In [1]:
import re
import math
import random
import time
import nltk

In [2]:
def sparseadd(onevec,othvec,weight=1):
    result={}
    try:
        for l in onevec:
            result[l] = onevec[l]
        for k in othvec:
            if k in result:
                result[k] = result[k]+othvec[k]*float(weight)
            else:
                result[k] = othvec[k]*float(weight)
    except:
        print("sparseadd(): error")
    return result
def sparsemultiply(onevec,othvec,weight=1):
    result={}
    try:
        for l in onevec:
            if l in othvec:
                result[l] = onevec[l]*othvec[l]*float(weight)
    except:
        print("sparsemultiply(): error ")
    return result
def sparsexor(onevec,othvec):
    result={}
    try:
        for l in range(len(onevec)):
            if ((l in onevec) and not (l in othvec)):
                result[l] = 1
            if (not (l in onevec) and (l in othvec)):
                result[l] = 1        
    except:
        print("sparsexor(): error ")
    return result

def newrandomvector(n,k):
    vec = {}
    if (k > 0):# no need to be careful about this, right? and k % 2 == 0):
        nonzeros = random.sample(list(range(n)),k)
        negatives = random.sample(nonzeros,k//2)
        for i in nonzeros:
            vec[str(i)] = 1;
        for i in negatives:
            vec[str(i)] = -1;
    return vec

def newoperator(n):
    k = n//10
    return newrandomvector(n,k)

def sparsecosine(xvec,yvec,truncate=True):
    x2 = 0;
    y2 = 0;
    xy = 0;
    try:
        for i in xvec:
            x2 = x2+xvec[i]*xvec[i]
    except:
        print("sparsecosine(): error at position ",i)
    try:
        for j in yvec:
            y2 = y2+yvec[j]*yvec[j]
            if j in xvec:
                xy = xy+xvec[j]*yvec[j]
    except:
        print("sparsecosine(): errors at position ",j)
    if (x2*y2 == 0):
        cos = 0
    else:
        cos = xy/(math.sqrt(x2)*math.sqrt(y2))
    if (truncate):
        cos=int(cos*10000)/10000
    return cos

def sparselength(vec,truncate=True):
    x2 = 0
    length=0
    try:
        for i in vec:
            x2 = x2+vec[i]*vec[i]
    except:
        print("sparselength(): error at position ",i)
    if (x2 > 0):
        length = math.sqrt(x2)
    if (truncate):
        length=int(length*10000)/10000
    return length

def comb(vec,k=0.1):
    newvector={}
    n=int(k*dimensionality/2)
    sorted_items=sorted(vec.items(), key=lambda x:x[1])
    bot=sorted_items[:n]
    top=sorted_items[-n:]
    for l in bot:
        newvector[l[0]]=l[1]
    for l in top:
        newvector[l[0]]=l[1]
    return newvector

def sparsesum(vec):
    s=0
    for i in vec:
        s += float(vec[i])
    return s

def normalise(vec,truncate=True):
    newvector={}
    vlen=sparselength(vec,False)
    if (vlen > 0):
        for i in vec:
            newvector[i]=vec[i]/math.sqrt(vlen*vlen)
    else:
        newvector=vec
    return newvector

def modify(vec,factor):
    newvector={}
    for i in vec:
        if (random.random() > factor):
            newvector[i]=vec[i]
        else:
            newvector[i]=float(vec[i])*(0.5-random.random())*2.0
    return newvector

def createpermutation(k):
    permutation=random.sample(range(k), k)
    return permutation
    
def permute(vector,permutation):
    newvector={}
    try:
        for i in range(len(permutation)):
            if str(i) in vector:
                newvector[str(permutation[i])]=vector[str(i)]
    except:
        newvector=vector
        print("permute(): no permutation done, something wrong")
    return newvector

In [28]:
wordspacefile="/home/jussi/Desktop/data/wordspaces/gavagai-news-22-morethan5.wordspace"
dimensionality=2000
denseness=10
fdgtextfile="/home/jussi/Desktop/data/GH95-fdg/950102.sgml.fdg.xml"
unittestfile="/home/jussi/Desktop/data/cooksents.txt"

In [56]:
definitepermutation=createpermutation(dimensionality)
indefinitepermutation=createpermutation(dimensionality)
futurepermutation=createpermutation(dimensionality)
pasttensepermutation=createpermutation(dimensionality)
presenttensepermutation=createpermutation(dimensionality)
agentpermutation=createpermutation(dimensionality)
patientpermutation=createpermutation(dimensionality)
eventpermutation=createpermutation(dimensionality)
instrumentpermutation=createpermutation(dimensionality)
locationpermutation=createpermutation(dimensionality)
mannerpermutation=createpermutation(dimensionality)
negationpermutation=createpermutation(dimensionality)
adverbialpermutation=createpermutation(dimensionality)

densenessforabstracts=dimensionality//10
agentabstract=newrandomvector(dimensionality,densenessforabstracts)
patientabstract=newrandomvector(dimensionality,densenessforabstracts)
eventabstract=newrandomvector(dimensionality,densenessforabstracts)
instrumentabstract=newrandomvector(dimensionality,densenessforabstracts)
locationabstract=newrandomvector(dimensionality,densenessforabstracts)
mannerabstract=newrandomvector(dimensionality,densenessforabstracts)

In [45]:
class clause:
    cleanuppattern=re.compile(r'[\.\'\!\*\?\+,;\:\-\/]+')
    def __init__(self,string):
        self.surfacestring=string
        self.cleanedutterance=re.sub(clause.cleanuppattern,"",string)
        self.tokens=self.cleanedutterance.lower().split()
        self.agent=None
        self.event=None
        self.patient=None
        self.instrument=None
        self.location=None
        self.manner=None
    def __str__(self):
        return self.surfacestring
    
class referent:
    def __init__(self, string):
        self.surfacestring=string
        self.definite=True
        self.number=1
        
class event:
    def __init__(self, string):
        self.surfacestring=string    
        self.negated=False    
        self.adverbial=None    
        self.tense=None #past present or future
        self.aspect=None #ongoing perfect or pointwise
        self.mood=None #indicative, irreal, potential, optative, not, imperative

In [46]:
from nltk.parse.stanford import StanfordDependencyParser
path_to_stanford_corenlp = '/usr/share/stanford-corenlp-full/'
parser_jar='stanford-corenlp.jar'
path_to_models = path_to_stanford_corenlp+'models/'
model_jar='stanford-english-corenlp-models.jar'
dependency_parser = StanfordDependencyParser(path_to_jar=path_to_stanford_corenlp+parser_jar, path_to_models_jar=path_to_models+model_jar)

def semanticdepparse(string,debug=False):
    utterance=clause(string)
    utterance.agent=referent("epsilon")
    utterance.patient=referent("epsilon")
    utterance.event=event("epsilon")
    depgraph=dependency_parser.raw_parse(string)
    scratch={}
    negated=False
    adverbial=None
    for deps in depgraph:
#        if debug:
#            print(deps)
        for nodeindex in deps.nodes:
            if deps.nodes[nodeindex]['rel'] == 'neg':
                negated=True
            elif deps.nodes[nodeindex]['rel'] == 'advmod':
                adverbial=deps.nodes[nodeindex]['word']
            elif deps.nodes[nodeindex]['rel'] == 'nsubj':
                utterance.agent=referent(deps.nodes[nodeindex]['word'])
                if nodeindex in scratch: 
                    if scratch[nodeindex]=="def":
                        utterance.agent.definite=True
                    elif scratch[nodeindex]=="indef":
                        utterance.agent.definite=False
            elif deps.nodes[nodeindex]['rel'] == 'nmod:poss':
                scratch[deps.nodes[nodeindex]['head']]="def"
            elif deps.nodes[nodeindex]['rel'] == 'det':
                if (deps.nodes[nodeindex]['word']=="a" or deps.nodes[nodeindex]['word']=="an"):
                    scratch[deps.nodes[nodeindex]['head']]="indef"
                elif (deps.nodes[nodeindex]['word']=="the"):
                    scratch[deps.nodes[nodeindex]['head']]="def"
            elif deps.nodes[nodeindex]['rel'] == 'root':
                utterance.event=event(deps.nodes[nodeindex]['word'])
            elif deps.nodes[nodeindex]['rel'] == 'dobj':
                utterance.patient=referent(deps.nodes[nodeindex]['word'])
                if nodeindex in scratch: 
                    if scratch[nodeindex]=="def":
                        utterance.patient.definite=True
                    elif scratch[nodeindex]=="indef":
                        utterance.patient.definite=False
        try:
            utterance.event.negated=negated
        except:
            print("semanticdepparse(): negation mismatch in "+string)
        try:
            utterance.event.adverbial=adverbial
        except:
            print("semanticdepparse(): adverbial blowout in "+string)
    if debug:
            print("semanticdepparse(): e:",utterance.event.surfacestring," a:",utterance.agent.surfacestring," p:",utterance.patient.surfacestring)
    return utterance


In [10]:
vectorspace={}
globalfrequency={}
indexspace={}
associationspace={}

def chkwordspace(words,debug=False):
    i = 0
    for w in words:
        if w in vectorspace:
                i += 1
        else:
            if debug:
                print("chkwordspace(): ",w," is new and is now hallucinated.")
            denseness2=dimensionality//2
            vectorspace[w] = newrandomvector(dimensionality,denseness2)
            indexspace[w] = newrandomvector(dimensionality,denseness)
            associationspace[w] = newrandomvector(dimensionality,denseness2)    

chkwordspace(["epsilon"],True)

def wordlikeness(w1,w2):
    return sparsecosine(vectorspace[w1],vectorspace[w2])

def dowordspacefile(wordspacefile):
    linepattern = re.compile(r'^\(\"([\-\w]+)\"\s+#S2000([\d\,\-\.\-\+;e]+)*:\s+#S2000([\d\-\+;\.e\,]+)*:\s+#S2000([\.\d\-\+;e\,]+)*:\s+(\d+)\)')
    vecpattern = re.compile(r';?(\d+)([\+\-][\,\.\d]+(e-\d\d)?)')
    alicefile=open(wordspacefile,'r')
    global vectorspace
    vectorspace={}
    global globalfrequency
    globalfrequency={}
    global indexspace
    indexspace={}
    global associationspace
    associationspace={}
    aliceline = alicefile.readline()
    a=1
    b=0
    w=0
    while aliceline:
        a = a+1
        m=linepattern.match(aliceline)
        if m:
            token=m.groups()[0]
            ind=m.groups()[1]
            doc=m.groups()[2]
            ctx=m.groups()[3]
            freq=int(m.groups()[4])
#            if (filtering and token )
            try:
                sparsevec={}
                cells = re.findall(vecpattern,ctx)
                for c in cells:
                    val = c[1]
                    kop=val.translate(str.maketrans(",",".","+"))
                    try:
                        sparsevec[c[0]] = float(kop)
                    except:
                        print("weird number: ",kop)
                vectorspace[token] = sparsevec
                sparsevec={}
                cells = re.findall(vecpattern,ind)
                for c in cells:
                    val = c[1]
                    kop=val.translate(str.maketrans(",",".","+"))
                    try:
                        sparsevec[c[0]] = float(kop)
                    except:
                        print("weird number: ",kop)
                indexspace[token] = sparsevec
                globalfrequency[token] = freq
                sparsevec={}
                w=w+1
            except:
                print("fail: ",token," ",a)
        else:
            b=b+1
        aliceline = alicefile.readline()
    print(a," lines seen; ",w," items added ",b," items disregarded")
    alicefile.close()
    

chkwordspace():  epsilon  is new and is now hallucinated.


In [135]:
def utterancevector(utterance,debug=False,normalised=True,lexical=True,morphology=True,semanticroles=True,morphologyrole=True):
    baremorphology=morphology
    combinedmorphology=lexical
    uttvec={}
    if debug:
        debugvec={}
    if lexical:     # add in referent, using its context vector meaning more or less the concept it refers to
        if (utterance.patient):
            if normalised:
                uttvec=sparseadd(
                    uttvec,
                    normalise(
                        comb(
                            vectorspace[utterance.patient.surfacestring])))
            else:
                uttvec=sparseadd(uttvec,comb(vectorspace[utterance.patient.surfacestring]))
        if (utterance.event):
            if normalised:
                uttvec=sparseadd(uttvec,normalise(comb(vectorspace[utterance.event.surfacestring])))#,sparselength(vectorspace[utterance.verb]))
            else:
                uttvec=sparseadd(uttvec,comb(vectorspace[utterance.patient.surfacestring]))
        if (utterance.agent):
            if normalised:
                uttvec=sparseadd(uttvec,normalise(comb(vectorspace[utterance.agent.surfacestring])))#,sparselength(vectorspace[utterance.agent]))
            else:
                uttvec=sparseadd(uttvec,comb(vectorspace[utterance.agent.surfacestring]))
    if debug:
        print("utterancevec() lexicon: ",sparsecosine(uttvec,debugvec))
        debugvec=uttvec
    if morphology:     # add in morphological information about the roles of the referent
        if (utterance.agent):
            if (utterance.agent.definite):
                if normalised:
                    uttvec=sparseadd(
                        uttvec,
                        normalise(
                            permute(
                                indexspace[utterance.agent.surfacestring],
                                definitepermutation)))
                else:
                    uttvec=sparseadd(uttvec,permute(indexspace[utterance.agent.surfacestring],definitepermutation))
            else:
                if normalised:
                    uttvec=sparseadd(uttvec,normalise(permute(indexspace[utterance.agent.surfacestring],indefinitepermutation)))
                else:
                    uttvec=sparseadd(uttvec,permute(indexspace[utterance.agent.surfacestring],indefinitepermutation))
        if (utterance.patient):
            if (utterance.patient.definite):
                if normalised:
                    uttvec=sparseadd(
                        uttvec,
                        normalise(
                            permute(indexspace[utterance.patient.surfacestring],
                                    definitepermutation)))
                else:
                    uttvec=sparseadd(uttvec,permute(indexspace[utterance.patient.surfacestring],definitepermutation))
            else:
     #       uttvec=sparseadd(uttvec,normalise(permute(comb(vectorspace[utterance.patient.surfacestring]),indefinite)))  
                if normalised:
                    uttvec=sparseadd(uttvec,normalise(permute(indexspace[utterance.patient.surfacestring],indefinitepermutation)))
                else:
                    uttvec=sparseadd(uttvec,permute(indexspace[utterance.patient.surfacestring],indefinitepermutation))
        if (utterance.event):
            if (utterance.event.tense=="FUTURE"):
                if normalised:
                    uttvec=sparseadd(uttvec,normalise(permute(vectorspace[utterance.event.surfacestring],futuretensepermutation)))
                else:
                    uttvec=sparseadd(uttvec,permute(vectorspace[utterance.event.surfacestring],futuretensepermutation))
            else:
                if normalised:
                    uttvec=sparseadd(uttvec,normalise(permute(vectorspace[utterance.event.surfacestring],presenttensepermutation)) )  
                else:
                    uttvec=sparseadd(uttvec,permute(vectorspace[utterance.event.surfacestring],presenttensepermutation))
    if debug:
        print("utterancevec() morphology: ",sparsecosine(uttvec,debugvec))
        debugvec=uttvec
    if morphologyrole:    # add in morphological information about the roles, no account of referent
        if (utterance.agent):
            if (utterance.agent.definite):
                uttvec=sparseadd(uttvec,normalise(permute(agentabstract,definitepermutation)))
            else:
                uttvec=sparseadd(uttvec,normalise(permute(agentabstract,indefinitepermutation)))
        if (utterance.patient):
            if (utterance.patient.definite):
                uttvec=sparseadd(uttvec,normalise(permute(patientabstract,definitepermutation)))
            else:
                 uttvec=sparseadd(uttvec,normalise(permute(patientabstract,indefinitepermutation)))
        if (utterance.event):
            if (utterance.event.tense=="FUTURE"):
                uttvec=sparseadd(uttvec,normalise(permute(eventabstract,futuretensepermutation)))
            else:
                uttvec=sparseadd(uttvec,normalise(permute(eventabstract,presenttensepermutation)) )  
    if debug:
        print("utterancevec() morphologyrole: ",sparsecosine(uttvec,debugvec))
        debugvec=uttvec
    if semanticroles:
        if (utterance.patient):
            if normalised:
                uttvec=sparseadd(uttvec,permute(normalise(indexspace[utterance.patient.surfacestring]),patientpermutation))
            else:
                uttvec=sparseadd(uttvec,permute(indexspace[utterance.patient.surfacestring],patientpermutation))
        if (utterance.event):
            if normalised:
                uttvec=sparseadd(uttvec,permute(normalise(indexspace[utterance.event.surfacestring]),eventpermutation))
            else:
                uttvec=sparseadd(uttvec,permute(indexspace[utterance.event.surfacestring],eventpermutation))
        if (utterance.event.negated):
            if normalised:
                uttvec=sparseadd(uttvec,permute(normalise(indexspace[utterance.event.surfacestring]),negationpermutation))
            else:
                uttvec=sparseadd(uttvec,permute(indexspace[utterance.event.surfacestring],negationpermutation))
        if (utterance.event.adverbial):
            if normalised:
                uttvec=sparseadd(uttvec,permute(normalise(indexspace[utterance.event.adverbial]),adverbialpermutation))
            else:
                uttvec=sparseadd(uttvec,permute(indexspace[utterance.event.adverbial],adverbialpermutation))
        if (utterance.agent):
            if normalised:
                uttvec=sparseadd(uttvec,permute(normalise(indexspace[utterance.agent.surfacestring]),agentpermutation))
            else:
                uttvec=sparseadd(uttvec,permute(indexspace[utterance.agent.surfacestring],agentpermutation))
    if debug:
        print("utterancevec() semanticrole: ",sparsecosine(uttvec,debugvec))
        debugvec=uttvec
    return uttvec


In [12]:
dowordspacefile(wordspacefile)

fail:  848-4158   251
fail:  j3   1843
fail:  939-3090   5604
fail:  artefax   5692
fail:  259-0033   5979
fail:  409-8008   6611
fail:  789-3456   12498
fail:  thwang   14318
fail:  30-oct   14713
fail:  938-9828   14879
fail:  259-7800   18300
fail:  840-6118   25485
fail:  650-5112   25870
fail:  laserbits   26632
fail:  492-0465   27336
fail:  885-4900   27849
fail:  871-1052   29176
fail:  273-7000   34112
fail:  885-5348   34937
fail:  725-0300   35289
fail:  727-1648   36388
fail:  375-1000   38233
fail:  365-8086   38337
fail:  834-1485   39746
fail:  lany   48235
fail:  chv   54315
fail:  kph   77436
fail:  ochone   94206
fail:  4269   101773
fail:  01267   102457
fail:  -1230   102881
fail:  -430   109265
fail:  8504   112260
fail:  -530   112513
fail:  attys   112964
fail:  76696   117710
fail:  bedrms   119252
fail:  videobits   123916
138847  lines seen;  135758  items added  3050  items disregarded


In [57]:
utterancespace = {}
dependencyanalysisstore={}
def initunittest(debug=False):
    with open(unittestfile,"r") as sents:
        s = sents.readline().rstrip()
        i = 0
        while s:
            print(s)
            wds=nltk.word_tokenize(s.lower())
            chkwordspace(wds,debug)
            try:
                c = semanticdepparse(s.lower(),debug)
                dependencyanalysisstore[s]=c
                u = utterancevector(c,debug)
                utterancespace[s] = u
                i += 1
            except:
                print("****",s)
            s = sents.readline()
def rununittest():
    for v in utterancespace:
        print(v)
        for w in utterancespace:
            print("          ",sparsecosine(utterancespace[v],utterancespace[w])," ",w)

In [59]:
initunittest(True)

The cook makes a pie.
semanticdepparse(): e: makes  a: cook  p: pie
utterancevec() lexicon:  0.0
utterancevec() morphology:  0.7076
utterancevec() morphologyrole:  1.0
utterancevec() syntacticrole:  0.8182
A cook bakes a pie.

semanticdepparse(): e: bakes  a: cook  p: pie
utterancevec() lexicon:  0.0
utterancevec() morphology:  0.6837
utterancevec() morphologyrole:  0.9999
utterancevec() syntacticrole:  0.8231
The cook bakes the cake.

semanticdepparse(): e: bakes  a: cook  p: cake
utterancevec() lexicon:  0.0
utterancevec() morphology:  0.7017
utterancevec() morphologyrole:  1.0
utterancevec() syntacticrole:  0.8232
The cook makes the stew.

semanticdepparse(): e: makes  a: cook  p: stew
utterancevec() lexicon:  0.0
utterancevec() morphology:  0.7158
utterancevec() morphologyrole:  1.0
utterancevec() syntacticrole:  0.8207
The cook prepared fish.

semanticdepparse(): e: prepared  a: cook  p: fish
utterancevec() lexicon:  0.0
utterancevec() morphology:  0.7286
utterancevec() morphology

In [ ]:
utterancespace = {}
for i in dependencyanalysisstore:
    utterancespace[i]=utterancevector(dependencyanalysisstore[i], True)

In [60]:
rununittest()

A cook bakes a pie.

           0.9999   A cook bakes a pie.

           0.2192   The cook makes the stew.

           0.538   The cook does not bake a pie.

           0.8396   The cook probably bakes a pie.

           0.2001   The cook prepared the fish.

           0.2201   The cook does not prepare a fish.

           0.2053   The cook probably prepares the fish.

           0.2001   The cook prepared fish.

           0.2211   The cook probably prepares a fish.

           0.5648   The cook bakes the cake.

           0.2165   The cook prepared a fish.

           0.5476   The cook makes a pie.
The cook makes the stew.

           0.2192   A cook bakes a pie.

           1.0   The cook makes the stew.

           0.3598   The cook does not bake a pie.

           0.3116   The cook probably bakes a pie.

           0.3188   The cook prepared the fish.

           0.2927   The cook does not prepare a fish.

           0.3072   The cook probably prepares the fish.

           0.3188

In [115]:
c=semanticdepparse("The cat eats fish.")

In [125]:
u=utterancevector(c)

In [123]:
vectorspace[c.patient.surfacestring]

{'599': 31.2161,
 '1239': -40.7035,
 '889': -1.3539,
 '954': 20.8136,
 '259': 4.52909,
 '1187': -16.0684,
 '1790': -41.3121,
 '54': 9.54726,
 '1662': -3.77058,
 '331': -10.8802,
 '1773': -26.285,
 '1100': -0.738639,
 '1194': -20.0026,
 '144': 27.5312,
 '256': 6.90189,
 '521': 1.26178,
 '465': 5.17814,
 '24': 8.6721,
 '981': -13.2477,
 '821': -18.1422,
 '1983': 0.827538,
 '984': 137.21,
 '366': 10.7486,
 '369': 37.2503,
 '1931': 5.82433,
 '1960': -11.1356,
 '1425': -12.4555,
 '262': -46.6341,
 '1026': 10.1527,
 '1495': -16.3532,
 '26': 0.674334,
 '392': -0.91982,
 '1737': -23.1944,
 '171': 8.28291,
 '802': -33.8708,
 '1532': 11.5401,
 '1573': -6.66549,
 '617': -27.8444,
 '1398': 11.7392,
 '1797': -20.1659,
 '1980': 16.8598,
 '1957': -30.9232,
 '734': 17.9864,
 '431': 11.6512,
 '1407': -17.2175,
 '1150': -6.97618,
 '261': -7.77552,
 '1314': -45.7488,
 '1226': 8.78049,
 '917': -2.13917,
 '651': -8.37052,
 '371': -17.9453,
 '1578': 6.70309,
 '339': 12.4702,
 '993': -12.7433,
 '1584': -9.73

In [40]:
def showtree(sentence):
    depgraph=dependency_parser.raw_parse(sentence.lower())
    for deps in depgraph:
        print(deps)
        for nodeindex in deps.nodes:
            print(deps.nodes[nodeindex])

In [140]:
def command_loop(debug=False,moredebug=False):
    instring = ''
    prev=newrandomvector(dimensionality,denseness)
    prevlex=newrandomvector(dimensionality,denseness)
    prevsem=newrandomvector(dimensionality,denseness)
    prevmor=newrandomvector(dimensionality,denseness)
    prevmgn=newrandomvector(dimensionality,denseness)
    while instring != 'quit':
        try:
            instring = input('> ')
            s = instring.rstrip()
            wds=nltk.word_tokenize(s.lower())
            chkwordspace(wds,debug)
            try:
                c = semanticdepparse(s.lower(),debug)
                u = utterancevector(c,moredebug, True, True, False, False,False)
                print(" lexical     ",sparsecosine(prev,u),end="\t(")
                print(sparsecosine(prevlex,u),")")
                prevlex=u
                u = utterancevector(c,moredebug, True, False, False, True, False)
                print(" semrole     ",sparsecosine(prev,u),end="\t(")
                print(sparsecosine(prevsem,u),")")
                prevsem=u
                u = utterancevector(c,moredebug, True, False, True, False, False)
                print(" morph     ",sparsecosine(prev,u),end="\t(")
                print(sparsecosine(prevmor,u),")")
                prevmor=u
                u = utterancevector(c,moredebug, True, False, False, False, True)
                print(" morphgen  ",sparsecosine(prev,u),end="\t(")
                print(sparsecosine(prevmgn,u),")")
                prevmgn=u
                u = utterancevector(c,moredebug, True)
                print(" in toto     ",sparsecosine(prev,u))
                prev = u
            except:
                print("****")
        except:
            print("hey!")
            instring = 'quit'
    
    

In [141]:
command_loop(True,False)

> The cat eats the fish.
semanticdepparse(): e: eats  a: cat  p: fish
 lexical      0.0	(-0.0155 )
 semrole      0.0	(0.0 )
 morph      0.0399	(-0.0126 )
 morphgen   -0.0259	(-0.0518 )
 in toto      0.0069
> The cat eats the meat.
semanticdepparse(): e: eats  a: cat  p: meat
 lexical      0.4106	(0.693 )
 semrole      0.3376	(0.6666 )
 morph      0.326	(0.6713 )
 morphgen   0.5005	(1.0 )
 in toto      0.7682
> we eat the meat
semanticdepparse(): e: eat  a: we  p: meat
 lexical      0.2202	(0.4218 )
 semrole      0.1635	(0.3333 )
 morph      0.2536	(0.4389 )
 morphgen   0.501	(1.0 )
 in toto      0.5519
> we eat the fish
semanticdepparse(): e: eat  a: we  p: fish
 lexical      0.3455	(0.6908 )
 semrole      0.3634	(0.6666 )
 morph      0.3583	(0.6578 )
 morphgen   0.5142	(1.0 )
 in toto      0.7671
> we do not eat the fish
semanticdepparse(): e: eat  a: we  p: fish
 lexical      0.5247	(1.0 )
 semrole      0.4366	(0.866 )
 morph      0.5	(1.0 )
 morphgen   0.5206	(1.0 )
 in toto      0.

In [132]:
showtree("We eat the fish.".lower())

defaultdict(<function DependencyGraph.__init__.<locals>.<lambda> at 0x7fac3bff6048>,
            {0: {'address': 0,
                 'ctag': 'TOP',
                 'deps': defaultdict(<class 'list'>, {'root': [2]}),
                 'feats': None,
                 'head': None,
                 'lemma': None,
                 'rel': None,
                 'tag': 'TOP',
                 'word': None},
             1: {'address': 1,
                 'ctag': 'PRP',
                 'deps': defaultdict(<class 'list'>, {}),
                 'feats': '_',
                 'head': 2,
                 'lemma': '_',
                 'rel': 'nsubj',
                 'tag': 'PRP',
                 'word': 'we'},
             2: {'address': 2,
                 'ctag': 'VBP',
                 'deps': defaultdict(<class 'list'>,
                                     {'dobj': [4],
                                      'nsubj': [1]}),
                 'feats': '_',
                 'head': 0,
        